# Task for the identification of argumentative sentence types

Files needed to run the notebook:

- The pickle file *dataset.pkl*; 

Import libraries:

In [1]:
import pickle as pkl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import json
import numpy as np

from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.regularizers import l2

from yellowbrick.text import FreqDistVisualizer


In [2]:
df = pd.read_pickle("./dataset.pkl")  

In [3]:
df.head()

,Document,Name,Id,Sentence,Type,Supported_by,Supported_from_failure,Attacked_by,Inhibited_by,Rephrased_by,Argumentation_scheme
2,2004,prem,A3,respect borne mind pursuant Article 58 Statute...,L,NaN,NaN,NaN,NaN,NaN,Rule
3,2004,prem,A4,Advocate General states point 20 Opinion Commi...,F,NaN,NaN,NaN,NaN,NaN,Aut
5,2004,prem,A6,Regarding Wam’s argument Commission’s appeal i...,L|F,NaN,NaN,NaN,NaN,NaN,Rule
6,2004,prem,A7,Appeals judgments Court First Instance governe...,L,NaN,NaN,NaN,NaN,NaN,Rule
7,2004,prem,A8,Next must noted obligation provide statement r...,L,NaN,NaN,NaN,NaN,NaN,Prec


In [4]:
df["Argumentation_scheme"].values[0]

'Rule'

In [5]:
df["Argumentation_scheme"].value_counts()

Prec                  279
Itpr                   95
Princ                  64
Rule                   61
Prec|Rule              35
Itpr|Prec              22
Aut                    21
Class|Prec|Rule         9
Tele                    8
Class                   7
Class|Prec              5
Aut|Itpr                5
Prec|Tele               5
Itpr|Tele               4
Rule|Syst               3
Itpr|Rule               3
Prec|Syst               3
Syst                    3
Princ|Rule              2
Psy|Tele                2
Lit|Rule                2
Lit|Prec                2
Prec|Psy|Rule|Tele      1
Princ|Tele              1
Itpr|Syst               1
Itpr|Psy                1
Itpr|Lit                1
Aut|Prec|Princ          1
Princ|Psy|Syst          1
Itpr|Princ|Syst         1
Lit|Rule|Syst           1
Syst|Tele               1
Aut|Class               1
Lit|Rule|Tele           1
Aut|Itpr|Prec           1
Prec|Psy                1
Aut|Rule                1
Aut|Syst                1
Lit         

In [6]:
value_counts = df['Argumentation_scheme'].value_counts() # Specific column 
df_truncated60 = df[df['Argumentation_scheme'].isin(value_counts[value_counts > 60].index)]
df_truncated20 = df[df['Argumentation_scheme'].isin(value_counts[value_counts > 20].index)]

In [7]:
df_truncated60['Argumentation_scheme'].value_counts()

Prec     279
Itpr      95
Princ     64
Rule      61
Name: Argumentation_scheme, dtype: int64

In [8]:
df_truncated20['Argumentation_scheme'].value_counts()

Prec         279
Itpr          95
Princ         64
Rule          61
Prec|Rule     35
Itpr|Prec     22
Aut           21
Name: Argumentation_scheme, dtype: int64

In [9]:
df_balanced = pd.concat([df[df["Argumentation_scheme"] == "Prec"].sample(61),
                    df[df["Argumentation_scheme"] == "Itpr"].sample(61),
                    df[df["Argumentation_scheme"] == "Princ"].sample(61),
                    df[df["Argumentation_scheme"] == "Rule"]
                    ])

In [10]:
df_balanced["Argumentation_scheme"].value_counts()

Prec     61
Itpr     61
Princ    61
Rule     61
Name: Argumentation_scheme, dtype: int64

Choice of parameters:

In [11]:
# EMBEDDINGS (uncomment the chosen one)
# EMBEDDING = "bert_sentence"
EMBEDDING = "legal_bert_sentence"
# EMBEDDING = "tfidf"

# MODELS (uncomment the chosen one)
# MODEL = "SVC"
MODEL = "RNN"
# MODEL = "BERT"

# DATASET (uncomment the chosen one)
#DATASET = df (It does not work due to classes with single elements)
DATASET = df_truncated60
# DATASET = df_truncated20
# DATASET = df_balanced

TEST_SIZE = 0.15
VAL_SIZE = 0.2  # percentage taken from the train subset

# MODEL PARAMETERS

# RNN & BERT
EPOCHS = 140
BATCH_SIZE = 64
LR = 0.001
DROP = 0.3

# ONLY RNN PARAMETERS
L2_FACTOR = 0.001
LAYERS = [256, 128, 64] # 3 layers


# SVC
KERNEL = "poly" #{linear, poly, rbf, sigmoid’} 
GAMMA = 0.01
C =0.01
DEGREE = 5 # only for kernel "poly"

In [12]:
scheme_one_hot = pd.get_dummies(DATASET['Argumentation_scheme'])

df_new = pd.concat([DATASET['Sentence'],scheme_one_hot],axis=1)

In [13]:
df_new.head()

,Sentence,Itpr,Prec,Princ,Rule
2,respect borne mind pursuant Article 58 Statute...,0,0,0,1
5,Regarding Wam’s argument Commission’s appeal i...,0,0,0,1
6,Appeals judgments Court First Instance governe...,0,0,0,1
7,Next must noted obligation provide statement r...,0,1,0,0
11,According settled caselaw statement reasons re...,0,1,0,0


# Embedding

In [14]:
if EMBEDDING == "bert_sentence" or EMBEDDING == "legal_bert_sentence":

  if EMBEDDING == "bert_sentence":
    sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
  if EMBEDDING == "legal_bert_sentence":
    sbert_model = SentenceTransformer("nlpaueb/legal-bert-small-uncased")

  sentence_embeddings = sbert_model.encode(df_new["Sentence"].values)

  X = sentence_embeddings
  if MODEL == "RNN" or MODEL == "BERT":
    y = scheme_one_hot
  if MODEL == "SVC":
    y = DATASET['Argumentation_scheme'].values

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/nlpaueb_legal-bert-small-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
if EMBEDDING == "tfidf":
  #Convert sentences into vectors 
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(df_new["Sentence"].values)

  if MODEL == "RNN":
    y = scheme_one_hot
  if MODEL == "SVC":
    y = DATASET['Argumentation_scheme'].values

  # Embeddings with tfidf
  tfidf = TfidfTransformer()
  X = tfidf.fit_transform(X)
  X = X.toarray()

In [16]:
print("EMBEDDING: ", EMBEDDING)
print("SHAPE X: ", X.shape)
print("SHAPE y: ", y.shape)

EMBEDDING:  legal_bert_sentence
SHAPE X:  (499, 512)
SHAPE y:  (499, 4)


In [17]:
#Visualize word frequency
if EMBEDDING == "tfidf":
  features   = vectorizer.get_feature_names()
  visualizer = FreqDistVisualizer(features=features, size=(800, 1000))
  visualizer.fit(X)
  visualizer.finalize()

# Training

In [18]:
if MODEL == "RNN":
  OUTPUTS = y.shape[1]

  def create_deep_model(factor, rate):
      model = Sequential()      
      model.add(Dense(units=LAYERS[0],kernel_regularizer=l2(factor), 
        activation='relu')), Dropout(rate),
      model.add(Dense(units=LAYERS[1],kernel_regularizer=l2(factor),
        activation='relu')), Dropout(rate),
      model.add(Dense(units=LAYERS[2],kernel_regularizer=l2(factor),
        activation='relu')), Dropout(rate),
      #Output layer
      model.add(Dense(units=OUTPUTS, activation='softmax'))
      return model
  model= create_deep_model(factor=L2_FACTOR, rate=DROP)

  opt=tf.keras.optimizers.Adam(learning_rate=LR)
  model.compile(loss='categorical_crossentropy', optimizer=opt,   
  metrics=['accuracy'])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle= True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=VAL_SIZE, shuffle= True)

if MODEL == "SVC":
  from sklearn import svm
  #fit the model
  model = svm.SVC(kernel=KERNEL, degree=DEGREE, gamma=GAMMA, coef0=C)
  model.fit(X_train, y_train)

if MODEL == "RNN":
  #fit the model
  history=model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val), verbose=1)

if MODEL =="BERT":

  input = tf.keras.layers.Input(shape=(X_train.shape[-1]))
  drop = tf.keras.layers.Dropout(DROP, name="dropout")(input)
  output = tf.keras.layers.Dense(y_train.shape[-1], activation='sigmoid', name="output")(drop)
  model = tf.keras.Model(inputs=[input], outputs = [output])
  
  opt=tf.keras.optimizers.Adam(learning_rate=LR)
  model.compile(loss='binary_crossentropy', optimizer=opt,   
  metrics=['accuracy'])
  history=model.fit(x=X_train, y=y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val), verbose=1)

Epoch 1/140
6/6 [==============================] - 1s 53ms/step - loss: 1.8167 - accuracy: 0.4867 - val_loss: 1.6099 - val_accuracy: 0.5412
Epoch 2/140
6/6 [==============================] - 0s 15ms/step - loss: 1.5014 - accuracy: 0.6460 - val_loss: 1.4601 - val_accuracy: 0.5647
Epoch 3/140
6/6 [==============================] - 0s 9ms/step - loss: 1.2970 - accuracy: 0.6755 - val_loss: 1.3123 - val_accuracy: 0.6824
Epoch 4/140
6/6 [==============================] - 0s 13ms/step - loss: 1.1595 - accuracy: 0.7434 - val_loss: 1.2718 - val_accuracy: 0.7059
Epoch 5/140
6/6 [==============================] - 0s 10ms/step - loss: 1.0738 - accuracy: 0.8142 - val_loss: 1.3298 - val_accuracy: 0.6941
Epoch 6/140
6/6 [==============================] - 0s 10ms/step - loss: 0.9836 - accuracy: 0.7965 - val_loss: 1.1448 - val_accuracy: 0.7294
Epoch 7/140
6/6 [==============================] - 0s 9ms/step - loss: 0.9046 - accuracy: 0.8673 - val_loss: 1.1949 - val_accuracy: 0.7176
Epoch 8/140
6/6 [=====

# Evaluation

In [20]:
y_train.shape

(339, 4)

In [21]:
if MODEL == "RNN":
  y_test=y_test.values
  y_test=np.argmax(y_test, axis=1)
  y_pred = np.argmax(model.predict(X_test), axis=-1)
  from sklearn.metrics import classification_report
  print(classification_report(y_test, y_pred, zero_division=0, target_names=df_new.columns[1:]))

if MODEL == "SVC":
  #y_test=y_test.values
  #y_test=np.argmax(y_test, axis=1)
  y_pred = model.predict(X_test)
  from sklearn.metrics import classification_report
  print(classification_report(y_test, y_pred, zero_division=0, target_names=df_new.columns[1:]))

if MODEL =="BERT":

  y_test= y_test.values
  y_test=np.argmax(y_test, axis=1)
  y_pred = np.argmax(model.predict(X_test), axis=-1)
  from sklearn.metrics import classification_report
  print(classification_report(y_test, y_pred, zero_division=0, target_names=df_new.columns[1:]))

              precision    recall  f1-score   support

        Itpr       0.55      0.60      0.57        10
        Prec       0.89      0.92      0.90        51
       Princ       0.67      0.57      0.62         7
        Rule       0.80      0.57      0.67         7

    accuracy                           0.81        75
   macro avg       0.72      0.67      0.69        75
weighted avg       0.81      0.81      0.81        75



In [22]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 0, 2, 3, 1, 1, 1, 1, 1, 0, 2, 1,
       0, 1, 3, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 3, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 0, 2, 0, 1, 1, 1, 1, 0,
       1, 1, 2, 1, 1, 0, 1, 1, 1])

Saving of test parameters and results on file:

In [23]:
with open("argumentation_type_classification_results.txt", "a") as f:
    f.write("MODEL: " + MODEL + "\n")
    f.write("EMBEDDING: " + EMBEDDING  + "\n")
    f.write("\n")

    f.write("DATASET:\n")
    f.write(str(DATASET["Argumentation_scheme"].value_counts()))
    f.write("\n\n")
    f.write("NUMBER OF SAMPLES: " + str(len(X)) + "\n")
    f.write("TEST_SIZE: " + str(TEST_SIZE*100) + "%\n")
    f.write("\n")

    f.write("PARAMETERS:\n")

    if MODEL == "RNN":
      f.write("EPOCHS: " + str(EPOCHS) + "\n")
      f.write("BATCH_SIZE: " + str(BATCH_SIZE) + "\n")
      f.write("LAYERS: " + str(LAYERS) + "\n")
      f.write("LR: " + str(LR) + "\n")
      f.write("L2_FACTOR: " + str(L2_FACTOR) + "\n")
      f.write("DROP: " + str(DROP) + "\n")
      f.write("\n")

    if MODEL == "BERT":
      f.write("EPOCHS: " + str(EPOCHS) + "\n")
      f.write("BATCH_SIZE: " + str(BATCH_SIZE) + "\n")
      f.write("LR: " + str(LR) + "\n")
      f.write("DROP: " + str(DROP) + "\n")
      f.write("\n")

    if MODEL == "SVC":
      f.write("KERNEL: " + str(KERNEL) + "\n")
      f.write("GAMMA: " + str(GAMMA) + "\n")
      f.write("C: " + str(C) + "\n")
      f.write("DEGREE: " + str(DEGREE) + "\n")
      f.write("\n")

    f.write("RESULTS:\n")
    f.write(classification_report(y_test, y_pred, zero_division=0, target_names=df_new.columns[1:])+"\n")

    f.write("\n")
    f.write("#######################################################")
    f.write("\n\n")
    

